## Lab 1: Creating a simple customer support agent prototype

### Overview

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/) helps you deploying and operating AI agents securely at scale - using any framework and model. It provides you with the capability to move from prototype to production faster. 

In this 5-labs tutorial, we will demonstrate the end-to-end journey from prototype to production using a **Customer Support Agent**. For this example we will use [Strands Agents](https://strandsagents.com/latest/), a simple-to-use, code-first framework for building agents and the Anthropic Claude Sonnet 3.7 model from Amazon Bedrock. For your application you can use the framework and model of your choice. It's important to note that the concepts covered here can be applied using other frameworks and models as well.

**Workshop Journey:**
- **Lab 1 (Current)**: Create Agent Prototype - Build a functional customer support agent
- **Lab 2**: Enhance with Memory - Add conversation context and personalization
- **Lab 3**: Scale with Gateway & Identity - Share tools across agents securely
- **Lab 4**: Deploy to Production - Use AgentCore Runtime with observability
- **Lab 5**: Build User Interface - Create a customer-facing application

In this first lab, we'll build a Customer Support Agent prototype that will evolve throughout the workshop into a production-ready system serving multiple customers with persistent memory, shared tools, and full observability. Our agent will have the following local tools available:
- **get_return_policy()** - Get return policy for specific products
- **get_product_info()** - Get product information
- **web_search()** - Search the web for troubleshooting help
- **get_technical_support()** - Search a Bedrock Knowledge Base


### Architecture for Lab 1
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*Simple prototype running locally. In subsequent labs, we'll migrate this to AgentCore services with shared tools, persistent memory, and production-grade observability.*

### Prerequisites

* **AWS Account** with appropriate permissions
* **Python 3.10+** installed locally
* **AWS CLI configured** with credentials
* **Anthropic Claude 3.7** enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* **Strands Agents** and other libraries installed in the next cells
#### Not using an AWS workshop account? 

**Note:** If you are running this as a self-paced lab you must run create the cloudformation resources as shown in the workshop self-paced steps. If you have not, then uncomment and run the below code segment

In [1]:
#!bash scripts/prereq.sh

### Step 1: Install Dependencies and Import Libraries
Before we start, let's install the pre-requisites for this lab

In [2]:
# Install required packages
%pip install -U -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


We can now import the required libraries and initialize our boto3 session

In [3]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool

In [4]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

### Step 2: Implementing custom tools

Next, we will implement the 3 tools which will be provided to the Customer Support Agent.

Defining tools in Strands Agent is extremely simple, just add a `@tool` decorator to your function, and provide a description of the tool in the function's docstring. Strands Agents will use the function documentation, typing and arguments to provide context on this tool to your agent. 


#### Tool 1: Get Return Policy

**Purpose:** This tool helps customers understand return policies for different product categories. It provides detailed information about return windows, conditions, processes, and refund timelines so customers know exactly what to expect when returning items.

In [9]:
@tool
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information for a specific product category.

    Args:
        product_category: Electronics category (e.g., 'smartphones', 'laptops', 'accessories')

    Returns:
        Formatted return policy details including timeframes and conditions
    """
    # Mock return policy database - in real implementation, this would query policy database
    return_policies = {
        "smartphones": {
            "window": "30 days",
            "condition": "Original packaging, no physical damage, factory reset required",
            "process": "Online RMA portal or technical support",
            "refund_time": "5-7 business days after inspection",
            "shipping": "Free return shipping, prepaid label provided",
            "warranty": "1-year manufacturer warranty included",
        },
        "laptops": {
            "window": "30 days",
            "condition": "Original packaging, all accessories, no software modifications",
            "process": "Technical support verification required before return",
            "refund_time": "7-10 business days after inspection",
            "shipping": "Free return shipping with original packaging",
            "warranty": "1-year manufacturer warranty, extended options available",
        },
        "accessories": {
            "window": "30 days",
            "condition": "Unopened packaging preferred, all components included",
            "process": "Online return portal",
            "refund_time": "3-5 business days after receipt",
            "shipping": "Customer pays return shipping under $50",
            "warranty": "90-day manufacturer warranty",
        },
    }

    # Default policy for unlisted categories
    default_policy = {
        "window": "30 days",
        "condition": "Original condition with all included components",
        "process": "Contact technical support",
        "refund_time": "5-7 business days after inspection",
        "shipping": "Return shipping policies vary",
        "warranty": "Standard manufacturer warranty applies",
    }

    policy = return_policies.get(product_category.lower(), default_policy)
    return (
        f"Return Policy - {product_category.title()}:\n\n"
        f"• Return window: {policy['window']} from delivery\n"
        f"• Condition: {policy['condition']}\n"
        f"• Process: {policy['process']}\n"
        f"• Refund timeline: {policy['refund_time']}\n"
        f"• Shipping: {policy['shipping']}\n"
        f"• Warranty: {policy['warranty']}"
    )


print("✅ Return policy tool ready")

✅ Return policy tool ready


#### Tool 2: Get Product Information

**Purpose:** This tool provides customers with comprehensive product details including warranties, available models, key features, shipping policies, and return information. It helps customers make informed purchasing decisions and understand what they're buying.

In [10]:
@tool
def get_product_info(product_type: str) -> str:
    """
    Get detailed technical specifications and information for electronics products.

    Args:
        product_type: Electronics product type (e.g., 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        Formatted product information including warranty, features, and policies
    """
    # Mock product catalog - in real implementation, this would query a product database
    products = {
        "laptops": {
            "warranty": "1-year manufacturer warranty + optional extended coverage",
            "specs": "Intel/AMD processors, 8-32GB RAM, SSD storage, various display sizes",
            "features": "Backlit keyboards, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux support varies by model",
            "support": "Technical support and driver updates included",
        },
        "smartphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "5G/4G connectivity, 128GB-1TB storage, multiple camera systems",
            "features": "Wireless charging, water resistance, biometric security",
            "compatibility": "iOS/Android, carrier unlocked options available",
            "support": "Software updates and technical support included",
        },
        "headphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "Wired/wireless options, noise cancellation, 20Hz-20kHz frequency",
            "features": "Active noise cancellation, touch controls, voice assistant",
            "compatibility": "Bluetooth 5.0+, 3.5mm jack, USB-C charging",
            "support": "Firmware updates via companion app",
        },
        "monitors": {
            "warranty": "3-year manufacturer warranty",
            "specs": "4K/1440p/1080p resolutions, IPS/OLED panels, various sizes",
            "features": "HDR support, high refresh rates, adjustable stands",
            "compatibility": "HDMI, DisplayPort, USB-C inputs",
            "support": "Color calibration and technical support",
        },
    }
    product = products.get(product_type.lower())
    if not product:
        return f"Technical specifications for {product_type} not available. Please contact our technical support team for detailed product information and compatibility requirements."

    return (
        f"Technical Information - {product_type.title()}:\n\n"
        f"• Warranty: {product['warranty']}\n"
        f"• Specifications: {product['specs']}\n"
        f"• Key Features: {product['features']}\n"
        f"• Compatibility: {product['compatibility']}\n"
        f"• Support: {product['support']}"
    )


print("✅ get_product_info tool ready")

✅ get_product_info tool ready


#### Tool 3: Web-search

**Purpose:** This tool allows customers to get troubleshooting support or suggestions on product recommendations etc.

In [11]:
@tool
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.

    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.

    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"


print("✅ Web search tool ready")

✅ Web search tool ready


#### Customer Support Agent - Knowledge Base Integration Steps

##### Download product technical_support files from S3

In [12]:
import os


def download_files():
    # Get account and region
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    region = boto3.Session().region_name
    bucket_name = f"{account_id}-{region}-kb-data-bucket"

    # Create local folder
    os.makedirs("knowledge_base_data", exist_ok=True)

    # Download all files
    s3 = boto3.client("s3")
    objects = s3.list_objects_v2(Bucket=bucket_name)

    for obj in objects["Contents"]:
        file_name = obj["Key"]
        s3.download_file(bucket_name, file_name, f"knowledge_base_data/{file_name}")
        print(f"Downloaded: {file_name}")

    print("All files saved to: knowledge_base_data/")


# Run it
download_files()

Downloaded: laptop-maintenance-guide.txt
Downloaded: monitor-calibration-guide.txt
Downloaded: smartphone-setup-guide.txt
Downloaded: troubleshooting-guide.txt
Downloaded: warranty-service-guide.txt
Downloaded: wireless-connectivity-guide.txt
All files saved to: knowledge_base_data/


#### Knowledge Base Sync Job

##### Sync the knowledge base with product technical_support files from S3 which can be integrated with the agent

In [13]:
import time

# Get parameters
ssm = boto3.client("ssm")
bedrock = boto3.client("bedrock-agent")
s3 = boto3.client("s3")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")[
    "Parameter"
]["Value"]
ds_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/data-source-id")[
    "Parameter"
]["Value"]

# Get file names from S3 bucket
bucket_name = f"{account_id}-{region}-kb-data-bucket"
s3_objects = s3.list_objects_v2(Bucket=bucket_name)
file_names = [obj["Key"] for obj in s3_objects.get("Contents", [])]

# Start sync job
response = bedrock.start_ingestion_job(
    knowledgeBaseId=kb_id, dataSourceId=ds_id, description="Quick sync"
)

job_id = response["ingestionJob"]["ingestionJobId"]
print("Bedrock knowledge base sync job started, ingesting the data files from s3")

# Monitor until complete
while True:
    job = bedrock.get_ingestion_job(
        knowledgeBaseId=kb_id, dataSourceId=ds_id, ingestionJobId=job_id
    )["ingestionJob"]

    status = job["status"]

    if status in ["COMPLETE", "FAILED"]:
        break

    time.sleep(10)

# Print final result
if status == "COMPLETE":
    file_count = job.get("statistics", {}).get("numberOfDocumentsScanned", 0)
    files_list = ", ".join(file_names)
    print(
        f"Bedrock knowledge base sync job completed Successfully, ingested {file_count} files"
    )
    print(f"Files ingested: {files_list}")
else:
    print(f"Bedrock knowledge base sync job failed with status: {status}")

Bedrock knowledge base sync job started, ingesting the data files from s3
Bedrock knowledge base sync job completed Successfully, ingested 6 files
Files ingested: laptop-maintenance-guide.txt, monitor-calibration-guide.txt, smartphone-setup-guide.txt, troubleshooting-guide.txt, warranty-service-guide.txt, wireless-connectivity-guide.txt


#### Tool 4: Get Technical Support

**Purpose:**  This tool provides customers with comprehensive technical support and troubleshooting assistance by accessing our knowledge base of electronics documentation. It includes detailed setup guides, maintenance instructions, troubleshooting steps, connectivity solutions, and warranty service information. This tool helps customers resolve technical issues, properly configure their devices, and understand maintenance requirements for optimal product performance.

In [14]:
from strands.models import BedrockModel
from strands import Agent
from strands_tools import retrieve


@tool
def get_technical_support(issue_description: str) -> str:
    try:
        # Get KB ID from parameter store
        ssm = boto3.client("ssm")
        account_id = boto3.client("sts").get_caller_identity()["Account"]
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")[
            "Parameter"
        ]["Value"]
        print(f"Successfully retrieved KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "tech_support_query",
            "input": {
                "text": issue_description,
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4,
            },
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access technical support documentation. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in get_technical_support: {str(e)}")
        return f"Unable to access technical support documentation. Error: {str(e)}"


print("✅ Technical support tool ready")

✅ Technical support tool ready


### Step 4: Create and Configure the Customer Support Agent

Next, we will create the Customer Support Agent providing a model, the list of tools implemented in the previous step, and with a system prompt.

In [15]:
SYSTEM_PROMPT = """You are a helpful and professional customer support assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Support the customer with technical information and product specifications, and maintenance questions
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following tools:
1. get_return_policy() - For warranty and return policy questions
2. get_product_info() - To get information about a specific product
3. web_search() - To access current technical documentation, or for updated information. 
4. get_technical_support() - For troubleshooting issues, setup guides, maintenance tips, and detailed technical assistance
For any technical problems, setup questions, or maintenance concerns, always use the get_technical_support() tool as it contains our comprehensive technical documentation and step-by-step guides.

Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about electronic products or specifications."""

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.3,
    region_name=region,
)

# Create the customer support agent with all tools
agent = Agent(
    model=model,
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search,  # Tool 3: Access the web for updated information
        get_technical_support,  # Tool 4: Technical support & troubleshooting
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("Customer Support Agent created successfully!")

Customer Support Agent created successfully!


### Step 5: Test the Customer Support Agent

Let's test our agent with sample queries to ensure all tools work correctly.

#### Test Return check

In [16]:
response = agent("What's the return policy for my thinkpad X1 Carbon?")

I'd be happy to provide you with information about the return policy for your ThinkPad X1 Carbon. Let me look that up for you.
Tool #1: get_return_policy
Thank you for your inquiry about the return policy for your ThinkPad X1 Carbon. Here's the information you need:

**Return Policy for Your ThinkPad X1 Carbon:**
- You have a 30-day return window from the date of delivery
- The laptop must be returned in its original packaging with all accessories included
- No software modifications should be made to the device
- Before processing a return, our technical support team will need to verify the condition
- Once approved and inspected, you'll receive your refund within 7-10 business days
- Return shipping is free when using the original packaging
- Your laptop comes with a 1-year manufacturer warranty, with options to extend

Is there anything specific about the return process you'd like to know more about? Or perhaps you're experiencing an issue with your ThinkPad that I could help troubl

In [17]:
response = agent("My laptop won't turn on, what should I check?")

I'm sorry to hear your ThinkPad X1 Carbon won't turn on. Let me help you troubleshoot this issue by getting some technical support information for you.
Tool #2: get_technical_support
Successfully retrieved KB ID: 0IK0YI6AX2
Based on the technical support information, here are several steps you should check when your ThinkPad X1 Carbon won't turn on:

### Immediate Steps to Try:
1. **Check power cable connections**
   - Ensure the power adapter is firmly connected to both the laptop and the wall outlet
   - Look for any visible damage to the cable or adapter

2. **Verify power outlet functionality**
   - Try plugging another device into the same outlet to confirm it's working
   - Try connecting your laptop to a different outlet

3. **Try a different power cable if available**
   - If you have access to another compatible ThinkPad charger, see if that works

4. **Check for physical damage**
   - Inspect the charging port for bent pins or debris
   - Look for any signs of liquid damage o

#### Test Troubleshooting

In [18]:
response = agent(
    "I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?"
)

I understand your concern about your iPhone 14 heating up. This can be frustrating, but let me help you troubleshoot this issue. Let me get some technical support information for you.
Tool #3: get_technical_support
Successfully retrieved KB ID: 0IK0YI6AX2
Let me also check for more specific information about iPhones and their return policy:
Tool #4: get_product_info

Tool #5: get_return_policy
Thank you for sharing your concern about your iPhone 14 heating up. Since you purchased it last month, I have some options for you to consider:

### Troubleshooting Steps for iPhone 14 Overheating:

1. **Check for Background Apps**
   - Close unused apps running in the background
   - Check which apps are consuming the most battery (Settings > Battery)

2. **Update iOS Software**
   - Go to Settings > General > Software Update
   - Outdated software can cause performance issues and overheating

3. **Manage Location Services**
   - Some apps using constant location tracking can cause heating
   - 

## 🎉 Lab 1 Complete!

You've successfully created a functional Customer Support Agent prototype! Here's what you accomplished:

- Built an agent with 3 custom tools (return policy, product info, web search)  
- Tested multi-tool interactions and web search capabilities  
- Established the foundation for our production journey  

### Current Limitations (We'll fix these!)
- **Single user conversation memory** - local conversation session, multiple customers need multiple sessions.
- **Conversation history limited to session** - no long term memory or cross session information is available in the conversation.
- **Tools reusability** - tools aren't reusable across different agents  
- **Running locally only** - not scalable
- **Identity** - No user and/or agent identity or access control
- **Observability** - Limited observability into agent behavior
- **Existing APIs** - No access to existing enterprise APIs for customer data

##### Next Up [Lab 2: Personalize our agent by adding memory →](lab-02-agentcore-memory.ipynb)
